Next two cells have to be run

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import random, re, os
from tqdm import tqdm

In [3]:
sample_file_list = ['473_Westinghouse_WAB_FY2017_18.html',
 '156_Estee Lauder_EL_FY2020_21.html',
 '361_Principal Financial_PFG_FY2018_19.html',
 '334_NRG Energy_NRG_FY2020_21.html',
 '154_Equifax_EFX_FY2021_22.html',
 '386_Qorvo_QRVO_FY2017_18.html',
 '229_International business_IBM_FY2019_20.html',
 '179_Freeport_FCX_FY2022_23.html',
 '13_Automatic Data Processing_ADP_FY2017_18.html',
 '33_Amgen_AMG_FY2022_23.html'] # Originally chosen at random

folder= 'data/Redline/'

# Removing All Duplicates

## Version 1

In [23]:
# Step 1: Load the document
folder = 'data/redline/'
file_name = '1_Agilent Technology_A_FY2017_18.html'
with open(folder+file_name, 'r', encoding='utf-8') as file:
    document = file.read()

# Step 2: Parse the document
soup = BeautifulSoup(document, 'html.parser')

# Step 3: Find all <span> elements
span_elements = soup.find_all('span')

# Step 4: Extract information
data = []
for span in span_elements:
    text = span.get_text(strip=True)  # Extract the text inside the span
    attributes = span.attrs  # Get all attributes of the span
    
    # Extract specific attributes like 'class'
    class_attribute = attributes.get('class', None)
    other_attrs = {key: value for key, value in attributes.items() if key != 'class'}
    
    data.append({
        'text': text,
        'class_attribute': class_attribute,
        'other_attributes': other_attrs
    })

# Step 5: Convert to DataFrame (optional)
df = pd.DataFrame(data)


In [24]:
df

,text,class_attribute,other_attributes
0,2018,"[redline-mod-original-rendered, redline-mod-or...",{'redline-text': '2017'}
1,2018,[redline-mod-new],{}
2,,[redline-delete],{'redline-text': 'and posted on its corporate ...
3,,[redline-delete],{'redline-text': 'and posted '}
4,,[redline-delete],{'redline-text': 'and post '}
...,...,...,...
9085,/s/,[redline-add],{}
9086,"20, 2018","[redline-mod-original-rendered, redline-mod-or...","{'redline-text': '21, 2017'}"
9087,"20, 2018",[redline-mod-new],{}
9088,111,"[redline-mod-original-rendered, redline-mod-or...",{'redline-text': '112'}


In [25]:
df.to_excel(f"data/redline_output/sample_output_{file_name}.xlsx")

### Observations after looking at 1_Agilent Technology_A_FY2017_18.html

We can solve the problem of duplicates by `redline-mod-new` class text that has the same `text` value as the `text` value of classes `redline-mod-original-rendered` and `redline-mod-original` before it. This is because the span with the same redline text is actually inside `redline-mod-new` class.

# Version 2

In [ ]:
# Step 1: Load the document
folder = 'data/redline/'
file_name = '1_Agilent Technology_A_FY2017_18.html'
with open(folder+file_name, 'r', encoding='utf-8') as file:
    document = file.read()

# Step 2: Parse the document
soup = BeautifulSoup(document, 'html.parser')

# Step 3: Find all <span> elements
span_elements = soup.find_all('span')

# Step 4: Extract information
data = []
previous_text = ''
previous_class = ''
for span in span_elements:
    text = span.get_text(strip=True)  # Extract the text inside the span
    attributes = span.attrs  # Get all attributes of the span
    
    # Extract specific attributes like 'class'
    class_attribute = attributes.get('class', None)
    other_attrs = {key: value for key, value in attributes.items() if key != 'class'}
    
    if text == previous_text and class_attribute == ['redline-mod-new'] and ('redline-mod-original' in previous_class): # note that no matter how many classes a span has, they are always in list format in the class_attribute column
        continue
    else:
        data.append({
            'text': text,
            'class_attribute': class_attribute,
            'other_attributes': other_attrs
        })
    
    previous_text = text
    previous_class = class_attribute

# Step 5: Convert to DataFrame (optional)
df = pd.DataFrame(data)


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [63]:
df

,text,class_attribute,other_attributes
0,2018,"[redline-mod-original-rendered, redline-mod-or...",{'redline-text': '2017'}
1,,[redline-delete],{'redline-text': 'and posted on its corporate ...
2,,[redline-delete],{'redline-text': 'and posted '}
3,,[redline-delete],{'redline-text': 'and post '}
4,¨,[redline-add],{}
...,...,...,...
4855,"December 20, 2018",[redline-add],{}
4856,Dow R. Wilson,[redline-add],{}
4857,/s/,[redline-add],{}
4858,"20, 2018","[redline-mod-original-rendered, redline-mod-or...","{'redline-text': '21, 2017'}"


In [64]:
df.to_excel(f"data/redline_output/sample_output_{file_name}_version2.xlsx")

In [55]:
df.class_attribute.iloc[0]

['redline-mod-original-rendered', 'redline-mod-original']

In [56]:
df

,text,class_attribute,other_attributes
0,2018,"[redline-mod-original-rendered, redline-mod-or...",{'redline-text': '2017'}
1,2018,[redline-mod-new],{}
2,,[redline-delete],{'redline-text': 'and posted on its corporate ...
3,,[redline-delete],{'redline-text': 'and posted '}
4,,[redline-delete],{'redline-text': 'and post '}
...,...,...,...
9085,/s/,[redline-add],{}
9086,"20, 2018","[redline-mod-original-rendered, redline-mod-or...","{'redline-text': '21, 2017'}"
9087,"20, 2018",[redline-mod-new],{}
9088,111,"[redline-mod-original-rendered, redline-mod-or...",{'redline-text': '112'}


So far (with first 24 lines of excel reviewed), duplicates were successfully removed.

# Version 3: Extracting redline-text (deleted text) as a separate column

In [69]:
# Step 1: Load the document
folder = 'data/redline/'
file_name = '1_Agilent Technology_A_FY2017_18.html'
with open(folder+file_name, 'r', encoding='utf-8') as file:
    document = file.read()

# Step 2: Parse the document
soup = BeautifulSoup(document, 'html.parser')

# Step 3: Find all <span> elements
span_elements = soup.find_all('span')

# Step 4: Extract information
data = []
previous_text = ''
previous_class = ''
for span in span_elements:
    text = span.get_text(strip=True)  # Extract the text inside the span
    attributes = span.attrs  # Get all attributes of the span
    
    # Extract specific attributes like 'class'
    class_attribute = attributes.get('class', None)
    other_attrs = {key: value for key, value in attributes.items() if key not in ['class','redline-text']}
    redline_text = attributes.get('redline-text', None)
    
    if text == previous_text and class_attribute == ['redline-mod-new'] and ('redline-mod-original' in previous_class): # note that no matter how many classes a span has, they are always in list format in the class_attribute column
        continue
    else:
        data.append({
            'text': text,
            'class_attribute': class_attribute,
            'redline-text':redline_text,
            'other_attributes': other_attrs
        })
    
    previous_text = text
    previous_class = class_attribute

# Step 5: Convert to DataFrame (optional)
df = pd.DataFrame(data)


In [70]:
df

,text,class_attribute,redline-text,other_attributes
0,2018,"[redline-mod-original-rendered, redline-mod-or...",2017,{}
1,,[redline-delete],"and posted on its corporate website, if any,",{}
2,,[redline-delete],and posted,{}
3,,[redline-delete],and post,{}
4,¨,[redline-add],None,{}
...,...,...,...,...
4855,"December 20, 2018",[redline-add],None,{}
4856,Dow R. Wilson,[redline-add],None,{}
4857,/s/,[redline-add],None,{}
4858,"20, 2018","[redline-mod-original-rendered, redline-mod-or...","21, 2017",{}


In [71]:
df.to_excel(f"data/redline_output/sample_output_{file_name}_version3.xlsx")

## Bigger sample check

In [80]:
sample_file_list

['468_Versign_VRSN_FY2021_22.html',
 '26_Allstate_ALL_FY2018_19.html',
 '45_Aptiv_APTV_FY2017_18.html',
 '124_Caesars_CZR_FY2022_23.html',
 '117_Cintas_CTAS_FY2020_21.html',
 '206_Goldman_GS_FY2020_21.html',
 '284_Southwest airline_LUV_FY2022_23.html',
 '160_EOG Resource_EOG_FY2017_18.html',
 '280_Alliant_LNT_FY2022_23.html',
 '422_Synchrone Financial_SYF_FY2022_23.html']

In [83]:
# Step 1: Load the document
folder = 'data/redline/'
file_list = [file for file in os.listdir(folder) if file!='.DS_Store']
sample_file_list = random.choices(file_list,k=10)
for file_name in tqdm(sample_file_list):
    with open(folder+file_name, 'r', encoding='utf-8') as file:
        document = file.read()

    # Step 2: Parse the document
    soup = BeautifulSoup(document, 'html.parser')

    # Step 3: Find all <span> elements
    span_elements = soup.find_all('span')

    # Step 4: Extract information
    data = []
    previous_text = ''
    previous_class = ''
    for span in span_elements:
        text = span.get_text(strip=True)  # Extract the text inside the span
        attributes = span.attrs  # Get all attributes of the span
        
        # Extract specific attributes like 'class'
        class_attribute = attributes.get('class', None)
        other_attrs = {key: value for key, value in attributes.items() if key not in ['class','redline-text']}
        redline_text = attributes.get('redline-text', None)
        
        if text == previous_text and class_attribute == ['redline-mod-new'] and ('redline-mod-original' in previous_class): # note that no matter how many classes a span has, they are always in list format in the class_attribute column
            continue
        else:
            data.append({
                'text': text,
                'class_attribute': class_attribute,
                'redline-text':redline_text,
                'other_attributes': other_attrs
            })
        
        previous_text = text
        previous_class = class_attribute

    # Step 5: Convert to DataFrame (optional)
    df = pd.DataFrame(data)

    df.to_excel(f"data/redline_output/sample_output_{file_name}_v3_for_check.xlsx")

100%|██████████| 10/10 [00:25<00:00,  2.59s/it]


Findings after checking sample output:

# Version 4: Locating Sections

In [117]:
# Step 1: Load the document
folder = 'data/redline/'
file_name = '1_Agilent Technology_A_FY2017_18.html'
with open(folder+file_name, 'r', encoding='utf-8') as file:
    document = file.read()

# Parse the document
soup = BeautifulSoup(document, 'html.parser')

# Find all elements
span_elements = soup.find_all('span')
all_divs = soup.find_all(['div', 'span'])

# Initialize variables
data = []
current_item = None

# Iterate through all elements to track the current "Item"
for element in all_divs:

    # Check for "Item" headings
    if (element.name == 'span' or element.name=='div') and element.get_text(strip=True).lower().startswith("item"):
        current_item = element.get_text(strip=True)

    # Process <span> elements
    elif element.name == 'span':
        text = element.get_text(strip=True)
        attributes = element.attrs
        class_attribute = attributes.get('class', None)
        redline_text = attributes.get('redline-text', None)
        other_attrs = {key: value for key, value in attributes.items() if key not in ['class', 'redline-text']}

        # Append data
        data.append({
            'text': text,
            'class_attribute': class_attribute,
            'redline_text': redline_text,
            'other_attributes': other_attrs,
            'item_heading_raw': current_item  # Add the current item heading
        })

# Convert to DataFrame
df = pd.DataFrame(data)

In [112]:
df[df.item_heading_raw=='Item 1.']

,text,class_attribute,redline_text,other_attributes,item_heading_raw


In [115]:
df[df['item_heading_raw'].notna()].head(40)

,text,class_attribute,redline_text,other_attributes,item_heading_raw
23,25,"[redline-mod-original-rendered, redline-mod-or...",24,{},Item 3
24,25,[redline-mod-new],None,{},Item 3
25,25,"[redline-mod-original-rendered, redline-mod-or...",24,{},Item 4
26,25,[redline-mod-new],None,{},Item 4
27,new product and service introductions and futu...,"[redline-mod-original-rendered, redline-mod-or...",our future effective tax rate and tax valuatio...,{},Item 15
28,new product and service introductions and futu...,[redline-mod-new],None,{},Item 15
29,,[redline-delete],"growth in our businesses, our investments, inc...",{},Item 15
30,"indemnification, the use of contract manufactu...","[redline-mod-original-rendered, redline-mod-or...","our financial results, our operating margin, o...",{},Item 15
31,"indemnification, the use of contract manufactu...",[redline-mod-new],None,{},Item 15
32,,[redline-delete],impairment of goodwill and other intangible a...,{},Item 15


In [118]:
# Look for them in `font` elements
# Step 1: Load the document
folder = 'data/redline/'
file_name = '1_Agilent Technology_A_FY2017_18.html'
with open(folder+file_name, 'r', encoding='utf-8') as file:
    document = file.read()

# Parse the document
soup = BeautifulSoup(document, 'html.parser')

# Find all elements
span_elements = soup.find_all('span')
all_divs = soup.find_all(['font', 'span'])

# Initialize variables
data = []
current_item = None

# Iterate through all elements to track the current "Item"
for element in all_divs:

    # Check for "Item" headings
    if element.name=='font' and element.get_text(strip=True).lower().startswith("item"):
        current_item = element.get_text(strip=True)

    # Process <span> elements
    elif element.name == 'span':
        text = element.get_text(strip=True)
        attributes = element.attrs
        class_attribute = attributes.get('class', None)
        redline_text = attributes.get('redline-text', None)
        other_attrs = {key: value for key, value in attributes.items() if key not in ['class', 'redline-text']}

        # Append data
        data.append({
            'text': text,
            'class_attribute': class_attribute,
            'redline_text': redline_text,
            'other_attributes': other_attrs,
            'item_heading_raw': current_item  # Add the current item heading
        })

# Convert to DataFrame
df_headings_from_font = pd.DataFrame(data)

In [123]:
df_headings_from_font[df_headings_from_font['item_heading_raw'].notna()].head(40)

,text,class_attribute,redline_text,other_attributes,item_heading_raw
23,25,"[redline-mod-original-rendered, redline-mod-or...",24,{},Item 3
24,25,[redline-mod-new],None,{},Item 3
25,25,"[redline-mod-original-rendered, redline-mod-or...",24,{},Item 4
26,25,[redline-mod-new],None,{},Item 4
27,new product and service introductions and futu...,"[redline-mod-original-rendered, redline-mod-or...",our future effective tax rate and tax valuatio...,{},Item 15
28,new product and service introductions and futu...,[redline-mod-new],None,{},Item 15
29,,[redline-delete],"growth in our businesses, our investments, inc...",{},Item 15
30,"indemnification, the use of contract manufactu...","[redline-mod-original-rendered, redline-mod-or...","our financial results, our operating margin, o...",{},Item 15
31,"indemnification, the use of contract manufactu...",[redline-mod-new],None,{},Item 15
32,,[redline-delete],impairment of goodwill and other intangible a...,{},Item 15


## Attempt to Account for Table of Contents section

In [133]:
# Step 1: Load the document
folder = 'data/redline/'
file_name = '1_Agilent Technology_A_FY2017_18.html'
with open(folder+file_name, 'r', encoding='utf-8') as file:
    document = file.read()

# Parse the document
soup = BeautifulSoup(document, 'html.parser')

# Find all elements
span_elements = soup.find_all('span')
all_elements = soup.find_all(['div', 'span', 'hr', 'font'])

# Initialize variables
data = []
current_item = None
in_contents_section=False

# Iterate through all elements to track the current "Item"
for element in all_elements:
    # Start of the "TABLE OF CONTENTS" section
    if (
        element.name == 'font'
        and "TABLE OF CONTENTS" in element.get_text(strip=True).upper()
    ):
        current_item = "contents"
        in_contents_section=True
    
    # End of the "Contents" section
    if element.name == 'hr' and element.get('style') == "page-break-after:always":
        if in_contents_section:
            in_contents_section=False
            current_item = None

    # Check for "Item" headings
    if element.name=='font' and element.get_text(strip=True).lower().startswith("item") and not in_contents_section:
        current_item = element.get_text(strip=True)

    # Process <span> elements
    elif element.name == 'span':
        text = element.get_text(strip=True)
        attributes = element.attrs
        class_attribute = attributes.get('class', None)
        redline_text = attributes.get('redline-text', None)
        other_attrs = {key: value for key, value in attributes.items() if key not in ['class', 'redline-text']}

        # Append data
        data.append({
            'text': text,
            'class_attribute': class_attribute,
            'redline_text': redline_text,
            'other_attributes': other_attrs,
            'item_heading_raw': current_item  # Add the current item heading
        })

# Convert to DataFrame
df_headings_from_font_and_accounting_for_contents = pd.DataFrame(data)

In [134]:
df_headings_from_font_and_accounting_for_contents[df_headings_from_font_and_accounting_for_contents['item_heading_raw'].notna()].head(50)

,text,class_attribute,redline_text,other_attributes,item_heading_raw
23,25,"[redline-mod-original-rendered, redline-mod-or...",24,{},contents
24,25,[redline-mod-new],None,{},contents
25,25,"[redline-mod-original-rendered, redline-mod-or...",24,{},contents
26,25,[redline-mod-new],None,{},contents
27,new product and service introductions and futu...,"[redline-mod-original-rendered, redline-mod-or...",our future effective tax rate and tax valuatio...,{},contents
28,new product and service introductions and futu...,[redline-mod-new],None,{},contents
29,,[redline-delete],"growth in our businesses, our investments, inc...",{},contents
30,"indemnification, the use of contract manufactu...","[redline-mod-original-rendered, redline-mod-or...","our financial results, our operating margin, o...",{},contents
31,"indemnification, the use of contract manufactu...",[redline-mod-new],None,{},contents
32,,[redline-delete],impairment of goodwill and other intangible a...,{},contents


Conclusion: not realistic to do because of varying formats

In [173]:
# Extract `before` and `after` texts
def extract_before_after_from_div(parent_div):
    # Clone the div for manipulation
    before_clone = BeautifulSoup(str(parent_div), 'html.parser')
    after_clone = BeautifulSoup(str(parent_div), 'html.parser')

    # Process "before" content
    # Replace `redline-mod-new` text with its parent's `redline-text`
    for mod_original in before_clone.find_all('span', class_='redline-mod-original'):
        if 'redline-text' in mod_original.attrs:
            mod_original.string = mod_original['redline-text']

    # Remove `redline-add` elements for "before"
    for added_span in before_clone.find_all('span', class_='redline-add'):
        added_span.decompose()

    # Process "after" content
    # Keep `redline-mod-new` text but remove `redline-text` attributes from `redline-mod-original`
    for mod_original in after_clone.find_all('span', class_='redline-mod-original'):
        if 'redline-text' in mod_original.attrs:
            del mod_original['redline-text']

    # Extract plain text from the processed divs
    before_text = before_clone.get_text(strip=True)
    after_text = after_clone.get_text(strip=True)

    return before_text, after_text


for file_name in tqdm(sample_file_list):
    with open(folder+file_name, 'r', encoding='utf-8') as file:
        document = file.read()

    # Parse the document
    soup = BeautifulSoup(document, 'html.parser')

    # Find all elements
    span_elements = soup.find_all('span')
    all_divs = soup.find_all(['div','font', 'span'])

    # Initialize variables
    data = []
    current_item = None
    previous_text=''
    previous_class = []
    previous_redline_text=''

    # Iterate through all elements to track the current "Item"
    for element in all_divs:

        # Check for "Item" headings
        if ((element.name=='font' or element.name=='span') and element.get_text(strip=True).lower().startswith("item")):
            current_item = element.get_text(strip=True)

        # Process <span> elements
        elif element.name == 'span':
            text = element.get_text(strip=True)
            attributes = element.attrs
            class_attribute = attributes.get('class', None)
            redline_text = attributes.get('redline-text', None)
            other_attrs = {key: value for key, value in attributes.items() if key not in ['class', 'redline-text']}

            try: 
                if text == previous_text and class_attribute == ['redline-mod-new'] and ('redline-mod-original' in previous_class): # note that no matter how many classes a span has, they are always in list format in the class_attribute column
                    continue
                if 'redline-mod-original-rendered' not in class_attribute and 'redline-mod-original' in class_attribute: # This removes all repeating redline texts (more than one added text being linked to the same deleted/redline text) so that there are less modifications
                    continue
                else:
                    # Extract "before" and "after" from the parent div
                    before, after = extract_before_after_from_div(element.find_parent('div') or element)
                    data.append({
                        'added_text': text,
                        'class_attribute': class_attribute,
                        'deleted_text':redline_text,
                        'other_attributes': other_attrs,
                        'section_raw':current_item,
                        'before':before,
                        'after':after
                    })
            except TypeError:
                continue

        previous_text=text
        previous_class = class_attribute

    # Convert to DataFrame
    df = pd.DataFrame(data)
    df = df[df['class_attribute'].notna()]
    df = df[(df['added_text']!='')&df['deleted_text']!='']

    df.to_excel(f"data/redline_output/sample_output_{file_name}_v4_for_check.xlsx")

100%|██████████| 10/10 [11:24<00:00, 68.42s/it] 


In [169]:
df.head(40)

,added_text,class_attribute,deleted_text,other_attributes,section_raw,before,after
0,2021,"[redline-mod-original-rendered, redline-mod-or...",2020,{},None,"<span style=""color:#000000;font-family:'Times ...","<span style=""color:#000000;font-family:'Times ..."
1,2021,"[redline-mod-original-rendered, redline-mod-or...",2020,{},None,"<span style=""color:#000000;font-family:'Times ...","<span style=""color:#000000;font-family:'Times ..."
2,$,"[redline-mod-original-rendered, redline-mod-or...","$42,640,586,293",{},None,"<span style=""color:#000000;font-family:'Times ...","<span style=""color:#000000;font-family:'Times ..."
3,"64,028,235,982",[redline-mod-new],None,{},None,"<span class=""redline-mod-original"" redline-tex...","<span class=""redline-mod-original"" redline-tex..."
4,"2022,","[redline-mod-original-rendered, redline-mod-or...","2021,",{},None,"<span style=""color:#000000;font-family:'Times ...","<span style=""color:#000000;font-family:'Times ..."
5,"623,207,437","[redline-mod-original-rendered, redline-mod-or...","624,518,873.",{},None,"<span style=""color:#000000;font-family:'Times ...","<span style=""color:#000000;font-family:'Times ..."
6,.,[redline-mod-new],None,{},None,"<span class=""redline-mod-original"" redline-tex...","<span class=""redline-mod-original"" redline-tex..."
7,2022,"[redline-mod-original-rendered, redline-mod-or...",2021,{},None,"<span style=""color:#000000;font-family:'Times ...","<span style=""color:#000000;font-family:'Times ..."
8,2021),"[redline-mod-original-rendered, redline-mod-or...",2020),{},None,"<span style=""color:#000000;font-family:'Times ...","<span style=""color:#000000;font-family:'Times ..."
9,Report—2021,"[redline-mod-original-rendered, redline-mod-or...",Report—2020,{},None,"<span style=""color:#000000;font-family:'Times ...","<span style=""color:#000000;font-family:'Times ..."


Note that because there is Contents section in the beginning, there may be misidentification of sections. 

(include an example to explain)

# After Meeting (version 5)
- Deleting numbers
- Relying on HTML documentation to consistenely decide between additions/deletions/modifications.

In [ ]:
version = 'v5'
from datetime import datetime

def is_date(text):
    """Check if the given text is a date."""
    date_formats = [
        "%B %d, %Y",  # e.g., "October 2, 2017"
        "%d %B %Y",   # e.g., "2 October 2017"
        "%Y-%m-%d",   # e.g., "2017-10-02"
    ]
    for date_format in date_formats:
        try:
            datetime.strptime(text, date_format)
            return True
        except ValueError:
            continue
    return False

# Define helper functions to detect numerical changes
def is_short_numerical_change(text):
    if not text:
        return False

    if text =='':
        return True
    
    if len(text.strip()) <4:
        return True

    if is_date(text.strip()):
        return True
    
    # Match numbers, percentages, or numbers written as words
    cleaned_text = text.replace(',', '').replace('$', '').replace('%', '').replace('.', '').replace(' ', '').replace('(', '').replace(')', '').replace(':','').replace(';','').replace('/','').replace('\\', '').replace("million", '').replace('billion', '')
    if cleaned_text.isdigit():
        return True

    numerical_patterns = [
        r'\b(million|billion|thousand|hundred)\b',  # numbers written as words
        r'\b(one|two|three|four|five|six|seven|eight|nine|ten)\b',  # small numbers written as words
        r'\W*'
    ]

    for pattern in numerical_patterns:
        if re.fullmatch(pattern, text.lower()):
            return True

    return False

# Extract `before` and `after` texts
def extract_before_after_from_div(parent_div):
    # Clone the div for manipulation
    before_clone = BeautifulSoup(str(parent_div), 'html.parser')
    after_clone = BeautifulSoup(str(parent_div), 'html.parser')

    # Process "before" content
    # Replace `redline-mod-new` text with its parent's `redline-text`
    for mod_original in before_clone.find_all('span', class_='redline-mod-original'):
        if 'redline-text' in mod_original.attrs:
            mod_original.string = mod_original['redline-text']

    # Remove `redline-add` elements for "before"
    for added_span in before_clone.find_all('span', class_='redline-add'):
        added_span.decompose()

    # Process "after" content
    # Keep `redline-mod-new` text but remove `redline-text` attributes from `redline-mod-original`
    for mod_original in after_clone.find_all('span', class_='redline-mod-original'):
        if 'redline-text' in mod_original.attrs:
            del mod_original['redline-text']

    # Extract plain text from the processed divs
    before_text = before_clone.get_text(strip=True)
    after_text = after_clone.get_text(strip=True)

    return before_text, after_text


for file_name in tqdm(sample_file_list):
    with open(folder+file_name, 'r', encoding='utf-8') as file:
        document = file.read()

    # Parse the document
    soup = BeautifulSoup(document, 'html.parser')

    # Find all elements
    span_elements = soup.find_all('span')
    all_divs = soup.find_all(['div','font', 'span'])

    # Initialize variables
    data = []
    current_item = None
    previous_text=''
    previous_class = []
    previous_redline_text=''

    # Iterate through all elements to track the current "Item"
    for element in all_divs:

        # Check for "Item" headings
        if ((element.name=='font' or element.name=='span') and element.get_text(strip=True).lower().startswith("item")):
            current_item = element.get_text(strip=True)

        # Process <span> elements
        elif element.name == 'span':
            added_text = element.get_text(strip=True)
            attributes = element.attrs
            class_attribute = attributes.get('class', None)
            redline_text = attributes.get('redline-text', None)
            other_attrs = {key: value for key, value in attributes.items() if key not in ['class', 'redline-text']}
            
            try: 
                if added_text == previous_text and class_attribute == ['redline-mod-new'] and ('redline-mod-original' in previous_class): # note that no matter how many classes a span has, they are always in list format in the class_attribute column
                    continue
                if 'redline-mod-original-rendered' not in class_attribute and 'redline-mod-original' in class_attribute: # This removes all repeating redline texts (more than one added text being linked to the same deleted/redline text) so that there are less modifications
                    continue
                if (not added_text or added_text.isspace()) and (not redline_text or redline_text.isspace()) :
                    continue
                if 'redline-mod-original-rendered' not in class_attribute and 'redline-mod-original' in class_attribute:
                    continue
                if (not is_short_numerical_change(added_text)) and (not is_short_numerical_change(redline_text)):
                    # Extract "before" and "after" from the parent div
                    before, after = extract_before_after_from_div(element.find_parent('div') or element)
                    data.append({
                        'class_attribute': class_attribute,
                        'added_text': added_text,
                        'deleted_text':redline_text,
                        'other_attributes': other_attrs,
                        'section':current_item,
                        'before':before,
                        'after':after
                    })
                    previous_text=added_text
                    previous_class = class_attribute
            except TypeError:
                continue

    # Convert to DataFrame
    df = pd.DataFrame(data)
    df = df[df['class_attribute'].notna()]
    df = df[(df['added_text']!='')&df['deleted_text']!='']

    df.to_excel(f"data/redline_output/sample_output_{file_name}_{version}_for_check.xlsx")

100%|██████████| 10/10 [03:48<00:00, 22.85s/it]
